In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

from keypoint_matching import KeypointMatcher
from RANSAC import ImageAlignment, project_2d_to_6d, project_6d_to_2d
from utils import show_two_images, show_single_image

In [ ]:
boat1 = cv2.imread('boat1.pgm', cv2.IMREAD_GRAYSCALE)
boat2 = cv2.imread('boat2.pgm', cv2.IMREAD_GRAYSCALE)
show_two_images(boat1, boat2, title="Sample pair of images.")

In [ ]:
# get matches
kp_matcher = KeypointMatcher(contrastThreshold=0.1, edgeThreshold=5)
matches, kp1, des1, kp2, des2 = kp_matcher.match(boat1, boat2, show_matches=True)

In [ ]:
image_alignment = ImageAlignment()
params = image_alignment.align(boat1, kp1, boat2, kp2, matches)

In [ ]:
params

In [ ]:
x2, y2

In [ ]:
A = project_2d_to_6d(np.array([[x1, y1]]))
x = params

b = np.dot(A, x)

In [ ]:
b

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, 10))

In [ ]:
colors[0]

In [ ]:
def show_transformed_points(img1, img2, X1, kp1, kp2, matches, num_to_show=100):

    H1, W1 = img1.shape
    H2, W2 = img2.shape
    img = np.hstack([img1, img2])

    random_matches = np.random.choice(matches, num_to_show)

    fig, ax = plt.subplots(1, 1, figsize=(18, 7))
    colors = cm.rainbow(np.linspace(0, 1, num_to_show))

    for i, match in enumerate(random_matches):

        # select a single match to visualize
        x1, y1 = kp1[match.queryIdx].pt
        x2, y2 = kp2[match.trainIdx].pt

        # get (x1, y1) transformed to (x1_transformed, y1_transformed)
        A = project_2d_to_6d(np.array([[x1, y1]]))
        (x1_transformed, y1_transformed) = np.dot(A, params)

        ax.imshow(img, cmap="gray")
        ax.axis("off")
        ax.scatter(x1_transformed + W1, y1_transformed, s=200, marker="x", color=colors[i])
        ax.plot((x1, x1_transformed + W1), (y1, y1_transformed), linestyle="--", color=colors[i], marker="o")

    plt.show()

In [ ]:
show_transformed_points(boat1, boat2, X1, kp1, kp2, matches, num_to_show=100)

In [ ]:
# select a single match to visualize
sel_match = matches[10]
x1, y1 = kp1[sel_match.queryIdx].pt
x2, y2 = kp2[sel_match.trainIdx].pt

# get (x1, y1) transformed to (x1_transformed, y1_transformed)
A = project_2d_to_6d(np.array([[x1, y1]]))
(x1_transformed, y1_transformed) = np.dot(A, params)

H1, W1 = boat1.shape
H2, W2 = boat1.shape

fig, ax = plt.subplots(1, 1, figsize=(18, 7))

img = np.hstack([boat1, boat2])
ax.imshow(img, cmap="gray")
ax.axis("off")
# ax.scatter(x1, y1, s=100, color="red", facecolor="none", linewidths=2.0)
# ax.scatter(x2 + W1, y2, s=100, color="red", facecolor="none", linewidths=2.0)
ax.scatter(x1_transformed + W1, y1_transformed, s=200, marker="x", color="red")
ax.plot((x1, x1_transformed + W1), (y1, y1_transformed), linestyle="--", color="red", marker="o")

plt.show()

In [ ]:
X.size

In [ ]:
sel_match = matches[100]

sel_kp1 = [kp1[sel_match.queryIdx]]
sel_kp1[0].size = 10
sel_kp2 = [kp2[sel_match.trainIdx]]

show_two_images(
    cv2.drawKeypoints(boat1, sel_kp1, color=(0, 0, 255), outImage=None),
    cv2.drawKeypoints(boat2, sel_kp2, color=(0, 0, 255), outImage=None),
)

In [ ]:
sel_kp2

In [ ]:
boat1.shape

In [ ]:
cv2.KeyPoint?

In [ ]:
key_points = [cv2.KeyPoint(100, 250, size=10)]

In [ ]:
img = cv2.drawKeypoints(boat1, key_points, outImage=None, color=(0, 0, 255))

In [ ]:
cv2.drawKeypoints?

In [ ]:
show_single_image(img, figsize=(10, 8))

In [ ]:

# get matches
kp_matcher = KeypointMatcher(contrastThreshold=0.1, edgeThreshold=5)
matches, kp1, des1, kp2, des2 = kp_matcher.match(boat1, boat2, show_matches=True)

image_alignment = ImageAlignment()
params = image_alignment.align(boat1, kp1, boat2, kp2, matches)